## 4. 테이블정보 수집하기
1. 라이브러리 불러오기
2. 웹페이지 가져오기
3. 가져온 웹페이지 자료를 HTML형태로 추출
4. find, findAll로 정보 찾기
5. 찾은 정보를 데이터프레임으로 저장

In [ ]:
sparkkorea.com/테스트
표 데이터 외에 컬럼도 크롤링 해서
생성한다.

예)
columnList = 크롤링 한 값 (리스트)
pd.Dataframe(rowList, columns=columnList)


In [26]:
# 라이브러리 불러오기
import pandas as pd
import requests,bs4

# 데이터를 불러 올 웹페이지 url
url = "https://sparkkorea.com/%ED%85%8C%EC%8A%A4%ED%8A%B8/"

# url 내 웹페이지 데이터를 요청해서 가져오기
resp = requests.get(url)

# 웹페이지 데이터를 인코딩
resp.encoding='utf-8'

# 데이터를 텍스트화하기
data = resp.text

# 텍스트화한 데이터를 html형태로 추출하기
bs = bs4.BeautifulSoup(data,"html.parser")

# 테이블 찾기 -> tableTag 
tableTag = bs.find(name = "table",
                  attrs = {"id":"test_table"})

# 테이블에 thead라는 태그 이름에서 th 태그를 모두 찾기 -> 테이블의 컬럼명(thTags)
thTags = tableTag.find(name = "thead").findAll(name = "th")

# 테이블에 tbody라는 태그에서 tr 태그를 모두 찾기 -> 테이블 각 행의 내용(trTags)
trTags = tableTag.find(name = "tbody").findAll(name = "tr")
trTagsLen = len(trTags)

# 컬럼의 이름을 담을 빈 리스트
columnList =[]

# 각 행을 담을 빈 리스트
row=[]

# trTags의 길이 만큼 동작 할 반복문 #1-> (0 - 1,2) (1 - 1,2) (2 - 1,2)
for j in range(0, trTagsLen):
    # 각 열의 내용을 담을 빈 리스트
    column=[]
    
    # trTags리스트에서 td태그 찾기 -> 예를 들어 trTags[0] 값에서 td 태그를 모두 찾는 것
    tdTags = trTags[j].findAll(name = "td")
    tdTagsLen = len(tdTags)
    
    # tdTags의 길이 만큼 동작 할 반복문 #2
    for i in range(0, tdTagsLen):
        # 컬럼 이름을 담은 리스트의 길이가 
        # thTags(컬럼명정보가 담긴 리스트) 길이와 같아지면 동작
        if len(columnList) == thTagsLen:
            # 내용이 담긴 tdTags를 텍스트화해서 column리스트에 추가
            column.append(tdTags[i].text)
        else:
            columnList.append(thTags[i].text)
            column.append(tdTags[i].text)
    # 반복문 #2에서 담은 column리스트를 row리스트에 추가
    row.append(column)
# 컬럼명이 columnList에 담긴 이름인 데이터프레임으로 저장
result = pd.DataFrame(row, columns=columnList)
result

,학번,이름
0,101,김효관
1,102,이순신
2,103,김어진


## 5. 셀레니움 활용 웹브라우저 자동화
#### 1. 라이브러리 선언 및 드라이버 설정
1. 셀레니움 라이브러리 설치 및 크롬 엔진 드라이버 다운
2. 라이브러리 선언
3. 드라이버 위치 -> 위치 빼고는 똑같음
4. 드라이버 옵션 설정
5. 웹 드라이버 정의


#### 2. 웹페이지 이동


In [1]:
# 라이브러리 선언
import bs4,requests
import pandas as pd
from selenium import webdriver
# 크롬드라이버 위치 및 드라이버 생성
driver_loc = "./chromedriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920*1080")
driver = webdriver.Chrome(driver_loc, options=options)

# 접속할 사이트
url = "https://sparkkorea.com/"

# driver를 이용해 사이트 접속
driver.get(url)

# 퀴즈 버튼의 xpath
quizMenuXpath = '//*[@id="menu-item-382"]/a'

# 퀴즈 버튼을 클릭하는 동작
driver.find_element_by_xpath(quizMenuXpath).click()

# 동작 후에 나타나는 현재 사이트 url
curentUrl = driver.current_url

# 웹페이지 요청해서 데이터 가져옴
rep = requests.get(curentUrl)
rep.encoding = 'utf-8'
# 데이터를 텍스트화
data = rep.text

# 텍스트화된 데이터를 html형태로 추출
bs = bs4.BeautifulSoup(data,'html.parser')

# html형태가 된 데이터에서 태그가 div이고 class속성이 class_spark_quiz를 찾음
divTag = bs.find(name = "div",
                 attrs = {"class" : "class_spark_quiz"})

# divTag에서 태그가 a인 것을 모두 찾아 리스트에 저장
aTags = divTag.findAll(name = "a")
aTagsLen = len(aTags)

# 각 행을 담을 빈 리스트
row = []

# aTags의 길이만큼 동작 할 반복문 -> aTag[0]의 사이트링크 정보와 텍스트 정보를 담는다
for i in range(0, aTagsLen):
    eachLink = aTags[i].attrs["href"]
    eachText = aTags[i].text
    
    # row리스트에 추가
    row.append([eachLink, eachText])
# 데이터프레임으로 저장
result = pd.DataFrame(data = row, columns=["링크","타이틀"])
result

,링크,타이틀
0,https://forms.gle/Fw49w9GhWQChDcZm7,6/13 Spark 심화과정
1,https://forms.gle/G4TcXm3fKuHLHA6D6,6/13 Spark 기본과정
2,https://forms.gle/M8gr1kC2ubA3UDVp8,6/18 Spark [MAP_FILTER]
3,https://forms.gle/h8w5mZ4MNaPLCPbi6,6/18 Spark GroupBy 심화
4,https://forms.gle/q5yL6QHfueDLM5w27,6/25 Spark RDD 실전 분석
5,https://forms.gle/Gxb4y6LfVYiaLu4M7,6/27 Spark RDD 실전 분석2


### 에러가 나는 이유?? -> 페이지소스가 자바스크립트로 나옴!!!

In [2]:
import bs4,requests
import pandas as pd
from selenium import webdriver


driver_loc = "./chromedriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920x1080")
driver = webdriver.Chrome(driver_loc, options=options)


driver.get("https://sparkkorea.com/")
import time
time.sleep(3)
quizMenuXpath = '//*[@id="menu-item-382"]/a'

driver.find_element_by_xpath(quizMenuXpath).click()
curentUrl = driver.current_url
data = driver.page_source
print(data)


bs = bs4.BeautifulSoup(data,'html.parser')
divTag = bs.find(name = "div",
                 attrs = {"class" : "class_spark_quiz"})

aTags = divTag.findAll(name = "a")
aTagsLen = len(aTags)
row = []
for i in range(0, aTagsLen):
    eachLink = aTags[i].attrs["href"]
    eachText = aTags[i].text
    row.append([eachLink, eachText])
result = pd.DataFrame(data = row, columns=["링크","타이틀"])
result.to_csv("./test.csv",encoding = 'ms949',index = False)

In [5]:
# 셀레니움 라이브러리 선언
from selenium import webdriver
# 크롬드라이버 위치 설정
driver_loc = "./chromedriver.exe"
# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920x1080")
# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

# URL로 원하는 웹페이지로 이동
sparkUrl = "https://www.sparkkorea.com"
driver.get(url = sparkUrl)

# 액션대상의 xPath 복사하여 '클릭' 액션 실행
quizMenu = '//*[@id="menu-item-382"]/a'
driver.find_element_by_xpath(quizMenu).click()

# 현재 페이지의 소스 가져오기
html = driver.page_source
print(html)

<html lang="ko-KR"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="http://gmpg.org/xfn/11">
<link rel="pingback" href="https://sparkkorea.com/xmlrpc.php">

<title>퀴즈 – 스파크 코리아</title>
<meta name="google-site-verification" content="QRZVgHsoL23RuwmgmXNuXfO-mUaxCkdvhP2rRvLT270">

<!-- Async WordPress.com Remote Login -->
<script id="wpcom_remote_login_js">
var wpcom_remote_login_extra_auth = '';
function wpcom_remote_login_remove_dom_node_id( element_id ) {
	var dom_node = document.getElementById( element_id );
	if ( dom_node ) { dom_node.remove(); }
}
function wpcom_remote_login_remove_dom_node_classes( class_name ) {
	var dom_nodes = document.querySelectorAll( '.' + class_name );
	for ( var i = 0; i < dom_nodes.length; i++ ) {
		dom_nodes[i].remove();
	}
}
function wpcom_remote_login_final_cleanup() {
	wpcom_remote_login_remove_dom_node_classes( "wpcom_remote_login_msg" );
	wpcom_remote_login_remove_dom_node_id

In [4]:
# Beautifulsoup 라이브러리 선언
import bs4
# 페이지 소스 정리
bs = bs4.BeautifulSoup(html, "html.parser")

# div태그로 원하는 정보 접근
divTag = bs.find(name = "div", attrs = {"id":"id_spark_quiz"})
# a태그로 퀴즈 url과 퀴즈 제목 추출
aTags = divTag.findAll(name = "a")

# 퀴즈url과 제목 리스트에 담기
aTagsLen = len(aTags) # 퀴즈 갯수
quizList = [] # 퀴즈 제목과 url 담을 빈리스트 생성
finalQuizList = [] # 최종결과([퀴즈제목, 퀴즈url]) 담을 빈리스트 생성
for i in range(0, aTagsLen):
    quizName = aTags[i].text # 퀴즈 제목
    quizUrl = aTags[i].attrs["href"] #퀴즈 url
    quizList = [quizName, quizUrl] # 리스트에 담기
    finalQuizList.append(quizList)    

# 데이터프레임 라이브러리 선언 및 출력
import pandas as pd
pd.DataFrame(finalQuizList, columns = ["퀴즈제목", "퀴즈 URL"])

AttributeError: 'NoneType' object has no attribute 'findAll'